In [1]:
import numpy as np
import scipy.io
from scipy import signal, interpolate
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# ========== Parameters ==========
fs = 1000
window_len = 100  # 100ms window
overlap = 50      # 50ms overlap
step = window_len - overlap
delay_steps = 3   # 150ms lag (3 windows)

# Frequency bands (Hz)
freq_bands = [
    (5, 15),    # Band 1
    (20, 25),   # Band 2
    (75, 115),  # Band 3
    (125, 160), # Band 4
    (160, 175)  # Band 5
]

# ========== GPU Setup ==========
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ========== Transformer Model ==========
class ECoGTransformer(nn.Module):
    def __init__(self, input_dim, output_dim=5, d_model=128, nhead=4, num_layers=3):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        encoder_layers = TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=512,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        # x shape: [batch_size, seq_len, input_dim]
        x = self.embedding(x)  # [batch_size, seq_len, d_model]
        x = self.transformer(x)  # [batch_size, seq_len, d_model]
        x = x.mean(dim=1)  # Global average pooling
        return self.fc(x)

# ========== Signal Processing ==========
def filter_data(raw_eeg, fs=1000):
    """Bandpass filter 1-200Hz"""
    b = signal.firwin(101, [1, 200], pass_zero='bandpass', fs=fs)
    return signal.lfilter(b, [1.0], raw_eeg, axis=0)

def extract_window_features(window, fs=1000):
    """Extract 6 features per channel (1 time + 5 freq bands)"""
    num_samples, num_channels = window.shape

    # 1. Time-domain feature: average voltage
    time_feat = np.mean(window, axis=0)  # [num_channels]

    # 2. Frequency-domain features
    freqs = np.fft.rfftfreq(num_samples, d=1/fs)
    fft_vals = np.abs(np.fft.rfft(window, axis=0))

    freq_feats = []
    for low, high in freq_bands:
        mask = (freqs >= low) & (freqs < high)
        if np.any(mask):
            band_energy = np.mean(fft_vals[mask], axis=0)  # [num_channels]
        else:
            band_energy = np.zeros(num_channels)
        freq_feats.append(band_energy)

    # Combine features: [num_channels, 6]
    return np.column_stack([time_feat] + freq_feats)

def get_windowed_features(raw_ecog, fs, window_len, overlap):
    """Extract features for all windows"""
    num_samples, num_channels = raw_ecog.shape
    step = window_len - overlap
    num_windows = (num_samples - overlap) // step

    all_features = []
    for i in tqdm(range(num_windows), desc="Extracting features"):
        start = i * step
        end = start + window_len
        window = raw_ecog[start:end, :]
        filtered = filter_data(window, fs=fs)
        features = extract_window_features(filtered, fs=fs)
        all_features.append(features.flatten())  # Flatten to [num_channels*6]

    return np.array(all_features)

def create_sequential_dataset(features, delay_steps):
    """Create input sequences with delay"""
    seq_features = []
    for i in range(delay_steps, len(features)):
        seq = features[i-delay_steps:i]
        seq_features.append(seq)
    return np.stack(seq_features)

# ========== Data Preparation ==========
def downsample_glove(glove, window_len, overlap):
    """Downsample glove data to match feature rate"""
    step = window_len - overlap
    return np.array([
        np.mean(glove[i*step : i*step + window_len], axis=0)
        for i in range((len(glove) - overlap) // step)
    ])

def interpolate_prediction(pred, original_len, step):
    """Upsample predictions to original rate"""
    x_old = np.arange(pred.shape[0]) * step
    x_new = np.arange(original_len)
    return np.column_stack([
        interpolate.CubicSpline(x_old, pred[:, i])(x_new)
        for i in range(pred.shape[1])
    ])

# ========== Training Pipeline ==========
def train_transformer(X_train, y_train, device):
    """Train transformer model"""
    # Convert to PyTorch tensors
    train_x = torch.FloatTensor(X_train).to(device)  # [batch, seq_len, features]
    train_y = torch.FloatTensor(y_train).to(device)
    dataset = TensorDataset(train_x, train_y)

    # Initialize model
    input_dim = X_train.shape[-1]
    model = ECoGTransformer(input_dim=input_dim).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

    # Training loop
    train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
    best_loss = float('inf')
    for epoch in tqdm(range(100), desc="Training"):
        model.train()
        epoch_loss = 0
        for x, y in train_loader:
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        scheduler.step(epoch_loss)
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model.state_dict(), 'best_model.pth')

    return model

# ========== Main Execution ==========
if __name__ == "__main__":
    # Load data
    train_data = scipy.io.loadmat('raw_training_data.mat')
    test_data = scipy.io.loadmat('leaderboard_data.mat')
    train_ecogs = [train_data['train_ecog'][i].item() for i in range(3)]
    train_gloves = [train_data['train_dg'][i].item() for i in range(3)]
    test_ecogs = [test_data['leaderboard_ecog'][i].item() for i in range(3)]

    # Process each subject
    all_preds = []
    for subj in range(3):
        print(f"\n=== Processing Subject {subj+1} ===")

        # Feature extraction
        feats = get_windowed_features(train_ecogs[subj], fs, window_len, overlap)
        feats = StandardScaler().fit_transform(feats)

        # Create sequential dataset with delay
        X_seq = create_sequential_dataset(feats, delay_steps)
        glove_down = downsample_glove(train_gloves[subj], window_len, overlap)
        y_train = glove_down[delay_steps:]  # Align with delayed features

        # Train Transformer
        model = train_transformer(X_seq, y_train, device)

        # Test prediction
        test_feats = get_windowed_features(test_ecogs[subj], fs, window_len, overlap)
        test_feats = StandardScaler().fit_transform(test_feats)
        X_test_seq = create_sequential_dataset(test_feats, delay_steps)

        with torch.no_grad():
            pred_50hz = model(torch.FloatTensor(X_test_seq).to(device)).cpu().numpy()

        # Interpolate to 1000Hz
        pred_1000hz = interpolate_prediction(
            pred_50hz,
            len(test_ecogs[subj]),
            step
        )
        all_preds.append(pred_1000hz)

    # Save results
    predicted_dg = np.empty((3,1), dtype=object)
    for i in range(3):
        predicted_dg[i,0] = all_preds[i]
    scipy.io.savemat('transformer_prediction.mat', {'predicted_dg': predicted_dg})
    print("\n✅ Predictions saved to transformer_prediction.mat")

Using device: cuda

=== Processing Subject 1 ===


Extracting features: 100%|██████████| 2949/2949 [00:03<00:00, 905.61it/s]



=== Processing Subject 2 ===


Extracting features: 100%|██████████| 2949/2949 [00:02<00:00, 1119.09it/s]



=== Processing Subject 3 ===


Extracting features: 100%|██████████| 2949/2949 [00:03<00:00, 978.45it/s]



✅ Predictions saved to transformer_prediction.mat


In [2]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

# Use same inputs as before
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_seq)):
    print(f"\n📂 Fold {fold+1}/{n_splits}")

    # Split into train/validation
    X_train_cv, X_val_cv = X_seq[train_idx], X_seq[val_idx]
    y_train_cv, y_val_cv = y_train[train_idx], y_train[val_idx]

    # Train model on this fold
    model_cv = train_transformer(X_train_cv, y_train_cv, device)

    # Predict on validation set
    with torch.no_grad():
        val_pred = model_cv(torch.FloatTensor(X_val_cv).to(device)).cpu().numpy()

    # Compute R² score
    r2 = r2_score(y_val_cv, val_pred, multioutput='variance_weighted')
    print(f"Fold {fold+1} R²: {r2:.4f}")
    scores.append(r2)

# Final cross-validation performance
print(f"\n✅ Average R² across {n_splits} folds: {np.mean(scores):.4f}")


📂 Fold 1/5


Training: 100%|██████████| 100/100 [01:11<00:00,  1.41it/s]


Fold 1 R²: 0.9009

📂 Fold 2/5


Training: 100%|██████████| 100/100 [01:05<00:00,  1.53it/s]


Fold 2 R²: 0.8961

📂 Fold 3/5


...

Fold 3 R²: 0.8951

📂 Fold 4/5


Training: 100%|██████████| 100/100 [01:03<00:00,  1.57it/s]


Fold 4 R²: 0.8898

📂 Fold 5/5


Training: 100%|██████████| 100/100 [01:05<00:00,  1.54it/s]

Fold 5 R²: 0.8955

✅ Average R² across 5 folds: 0.8955
